In [8]:
import numpy as np
import scipy.linalg as la   

# Normalization matrix 

In [9]:
def normalization_matrix(p):
    '''
    Input: p: Nx2 matrix
    Output: T, normalization matrix (in projective plane) 
    '''

    # Computation
    m = np.mean(p,axis=0)
    q = p - np.repeat(m[np.newaxis,:],N,axis=0)
    w = np.sqrt(np.sum(q**2,axis=1))
    scale = 1/np.mean(w)

    # Normalization matrix
    T = np.zeros((3,3))
    T[2,2] = 1
    T[0,0] = scale
    T[0,2] = -m[0]*scale
    T[1,1] = scale
    T[1,2] = -m[1]*scale
        
    return T
    

# Utilities

In [15]:
def projectivation(p):
    '''
    Input: p, Nxd matrix = N points in R^d
    Output: q, Nx(d+1) = N points in P^d
    '''
    N,d = p.shape
    q = np.ones((N,d+1))
    q[:,0:d] = p 
    return q 

def affinization(q):
    '''
    Input: q, Nx(d+1) = N points in P^d
    Output: p, Nxd matrix = N points in R^d
    '''
    N,d1 = q.shape
    d = d1-1
    p = np.ones((N,d))
    p = q[:,0:d]/q[:,d:d+1] 
    
    return p


# Computing homography

In [26]:
def homography2d(p1,p2):
    '''
    Input: p1,p2: Nx2 matrices
    Output: A, homography from p1 to p2 (in projective plane) 
    '''

    # Normalization matrices
    T1 = normalization_matrix(p1)
    T2 = normalization_matrix(p2)

    # Projective points
    p1j = projectivation(p1)
    p2j = projectivation(p2)
    
    # Normalized points
    p1jn = (T1@(p1j.T)).T
    p2jn = (T2@(p2j.T)).T

    # Homography computation relying on normalized points
    N = p1.shape[0]
    M = np.zeros((2*N,9))
    for i in range(N):
        l0 = np.concatenate([[0,0,0],-p2jn[i,2]*p1jn[i,:],p2jn[i,1]*p1jn[i,:]])
        l1 = np.concatenate([p2jn[i,2]*p1jn[i,:],[0,0,0],-p2jn[i,0]*p1jn[i,:]])
        M[2*i,:] = l0
        M[2*i+1,:] = l1

    W = M.T@M
    U,D,Vt = la.svd(W,compute_uv=True)
    a = U[:,-1]
    A = np.reshape(a.T,(3,3))

    # Returning to non-normalized points
    A = la.inv(T2)@A@T1

    return A
    




# Experiment

In [101]:
N = 30
p1 = np.random.randint(0,500,(N,2))
A = np.random.randint(-5,5,(3,3))
p1j = projectivation(p1)
p2j = (A@p1j.T).T  
p2 = affinization(p2j)


noise_level1 = 0.5
noise_level2 = 0.01
p1 = np.float64(p1)
p1 += noise_level1*np.random.randn(N,2)
p2 += noise_level2*np.random.randn(N,2)





In [102]:
B = homography2d(p1,p2)
    

In [103]:
A


array([[-4,  2,  0],
       [-4,  1, -3],
       [ 4, -4, -1]])

In [104]:
B

array([[ 0.00342853, -0.00172307, -0.00532114],
       [ 0.00343045, -0.00084391, -0.00411197],
       [-0.00342159,  0.00340479,  0.00682051]])

In [105]:
A/B

array([[-1166.67938412, -1160.71566711,    -0.        ],
       [-1166.02675789, -1184.95854498,   729.57740358],
       [-1169.0469536 , -1174.8170285 ,  -146.616541  ]])

# Checking homography


In [83]:
def distance(p1,p2):
    '''
    Input: p1,p2: Nxd matrices = N points in R^d
    Output: d: Nx1 matrix = N distances 
    '''

    d = p2-p1
    d = np.sqrt(np.sum(d**2,axis=1))
    return d


def mapping_distance(A,p1,p2):
    '''
    Input: A: planar homography, i.e. 3x3 matrix
    p1,p2: Nx2 matrices, such that A maps p1 to p2 
    Ouput: Statistical measures of the mapping quality
    '''

    p1j = projectivation(p1)
    q2j = (A@p1j.T).T 
    q2 = affinization(q2j) 
    d = distance(q2,p2)

    return d


def check_homography2d(A,p1,p2):
    '''
    Input: A: planar homography, i.e. 3x3 matrix
    p1,p2: Nx2 matrices, such that A maps p1 to p2 
    Ouput: Statistical measures of the mapping quality
    '''

    d = mapping_distance(A,p1,p2)    
    m = np.mean(d)
    s = np.std(d) 
    return m,s




In [107]:
m,s = check_homography2d(A,p1,p2)
print('Average geometric error: ',m)
print('Std of geometric error: ',s)

Average geometric error:  0.039885599151868734
Std of geometric error:  0.07331747354954336


In [108]:
m,s = check_homography2d(B,p1,p2)
print('Average geometric error: ',m)
print('Std of geometric error: ',s)

Average geometric error:  0.037713091029255866
Std of geometric error:  0.0664052143435646


# Non-linear optimization

In [86]:
def f(x,p1,p2):
    A = np.reshape(x,(3,3))
    d = mapping_distance(A,p1,p2)
    return d

In [110]:
import scipy.optimize as opt
x0 = np.reshape(B,(9,1)).flatten()
res = opt.least_squares(f,x0,args=(p1,p2))


In [111]:
res


 active_mask: array([0., 0., 0., 0., 0., 0., 0., 0., 0.])
        cost: 0.07644742892834785
         fun: array([0.02761083, 0.01849445, 0.03110024, 0.00808393, 0.02935898,
       0.03159566, 0.01875111, 0.00865384, 0.00838118, 0.02707913,
       0.00966445, 0.01158508, 0.0295114 , 0.0233182 , 0.23527009,
       0.01518934, 0.01774684, 0.10867692, 0.27000701, 0.00639405,
       0.0079353 , 0.05239925, 0.01385483, 0.00234944, 0.02670206,
       0.01217434, 0.00582423, 0.03300915, 0.00605   , 0.01857032])
        grad: array([ -2.90005691,  -5.87384495,   0.1285165 ,   5.68000472,
         3.51031872,   0.18858496, -10.03900778,  -6.91188362,
        -3.31145503])
         jac: array([[-9.10257590e+02, -6.77916923e+02, -3.55549026e+00,
        -7.38586226e+02, -5.50199468e+02, -2.88534579e+00,
        -4.62292455e+03, -3.44164734e+03, -1.80533768e+01],
       [-9.23630386e+01, -3.02866913e+01, -2.30665111e-01,
        -4.31065966e+02, -1.41200066e+02, -1.07566778e+00,
        -7.05033067

In [112]:
res.x-x0
x0

array([ 0.00342853, -0.00172307, -0.00532114,  0.00343045, -0.00084391,
       -0.00411197, -0.00342159,  0.00340479,  0.00682051])

In [113]:
f(x0,p1,p2)

array([0.02470424, 0.01397332, 0.02028547, 0.01682633, 0.06599436,
       0.02872472, 0.00754842, 0.00951126, 0.00610849, 0.0168434 ,
       0.00943843, 0.00856857, 0.01630011, 0.01818759, 0.25449936,
       0.02183884, 0.01101184, 0.13654314, 0.28136023, 0.00591193,
       0.00760206, 0.03915164, 0.01015567, 0.00350127, 0.03069113,
       0.01615112, 0.0054428 , 0.02640249, 0.00460732, 0.01350717])

In [114]:
f(res.x,p1,p2)

array([0.02761083, 0.01849445, 0.03110024, 0.00808393, 0.02935898,
       0.03159566, 0.01875111, 0.00865384, 0.00838118, 0.02707913,
       0.00966445, 0.01158508, 0.0295114 , 0.0233182 , 0.23527009,
       0.01518934, 0.01774684, 0.10867692, 0.27000701, 0.00639405,
       0.0079353 , 0.05239925, 0.01385483, 0.00234944, 0.02670206,
       0.01217434, 0.00582423, 0.03300915, 0.00605   , 0.01857032])

In [99]:
p1

array([[313.92930822, 151.07720064],
       [487.97253581, 247.99557717],
       [ 90.91483202, 401.12349696],
       [106.0614229 , 393.88906206],
       [147.93295144, 144.03720925],
       [ 76.06870487, 450.89375788],
       [404.03383248,  32.12135473],
       [304.04729282,  31.12471899],
       [125.06928619, 416.9791105 ],
       [ 58.95822002, 484.83312186],
       [482.06874866, 211.91517154],
       [103.84920351, 143.99738598],
       [ 84.80175543, 240.98837216],
       [ 38.10975744, 356.05221649],
       [309.13525504, 236.99571037],
       [ 15.06701426, 143.10558667],
       [251.03351396, 299.89436782],
       [359.98521732, 121.99522226],
       [209.9642126 , 460.10869776],
       [ 82.01741727, 417.95854366],
       [186.07858369, 210.04470873],
       [ 33.02426591, 358.98581637],
       [ 52.93985052, 484.04530812],
       [432.97745725, 291.95031135],
       [351.99586014, 316.17017024],
       [334.96080528, 457.88997137],
       [353.94917139, 411.11564342],
 

In [100]:
p2

array([[  3.5073428 ,   0.78662628],
       [  3.84284304,   0.70713651],
       [ -4.65820383,   1.71233051],
       [ -5.32128973,   1.81400842],
       [  9.41837775,   0.13689434],
       [ -4.03652654,   1.60999177],
       [  1.86997583,   0.9492326 ],
       [  1.92144241,   0.98186912],
       [ -5.9905366 ,   2.01343905],
       [ -3.47443944,   1.64955798],
       [  3.42472799,   0.8207315 ],
       [ 47.81045661,  -4.83845916],
       [ -7.47444171,   2.11009687],
       [ -3.41115951,   1.67576217],
       [  5.92065034,   0.47132075],
       [ -3.49910998,   1.61256155],
       [ 17.83064408,  -0.85624059],
       [  2.85144393,   0.96759973],
       [-11.68949751,   2.74105572],
       [ -4.06775021,   1.64242273],
       [ 14.08462513,  -0.63144977],
       [ -3.26500384,   1.67957892],
       [ -3.35547588,   1.55332848],
       [  5.23032332,   0.54007911],
       [  7.93243584,   0.38477804],
       [ 43.63082181,  -4.11143541],
       [ 15.74256765,  -0.71436565],
 